In questo paper viene introdotto l'algoritmo di selective search (Uijlings et al. in 2012).<br>
Per mezzo di questa tecnica è possibile identificare automaticamente, all'interno di una immagine, le diverse posizioni dove possono trovarsi oggetti.<br>
E' molto più efficiente rispetto all'utilizzo di Image Pyramid e sliding windows.<br>
E' un algoritmo di "Automathic region proposal" ed è il predecessore degli algoritmi di RPN (Region Proposal Network) che sono ancora più accurati ed efficienti.

In [7]:
import random
import time
import cv2

In [15]:
IMAGE = "data/gatto_difficile.jpg"
METHOD = "fast"
#METHOD = "quality"

L'algoritmo di selective search effettua una over-segmentation dell'immagine utilizzando un algoritmo di "superpixel". <br>
I pixel che compongono l'immagine vengono aggregati sulla base di 5 misure di similarità:
- color similarity
- texture similarity
- size similarity (privilegiando in prima battuta le regioni di minori dimensioni)
- shape similarity (sulla base della compatibilità delle regioni proposte)
- un meta similarity measure che è dato dalla combinazione lineare delle precedenti misure


In [16]:
# load the input image
image = cv2.imread(IMAGE)
# initialize OpenCV's selective search implementation and set the
# input image
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss.setBaseImage(image)
# check to see if we are using the *fast* but *less accurate* version
# of selective search
if METHOD == "fast":
	print("[INFO] using *fast* selective search")
	ss.switchToSelectiveSearchFast()
# otherwise we are using the *slower* but *more accurate* version
else:
	print("[INFO] using *quality* selective search")
	ss.switchToSelectiveSearchQuality()

[INFO] using *fast* selective search


In [17]:
# run selective search on the input image
start = time.time()
rects = ss.process()
end = time.time()
# show how along selective search took to run along with the total
# number of returned region proposals
print("[INFO] selective search took {:.4f} seconds".format(end - start))
print("[INFO] {} total region proposals".format(len(rects)))

[INFO] selective search took 32.8505 seconds
[INFO] 11617 total region proposals


Una volta ottenute le regioni che presumibilmente contengono un oggetto, è possibile visualizzarle prima di passare alla successiva fase di classificazione.<br>
La visualizzazione avviene a gruppi di 100 regiorni individuate sul totale precedentemente estratto

In [19]:
# loop over the region proposals in chunks (so we can better
# visualize them)
for i in range(0, len(rects), 100):
    # clone the original image so we can draw on it
    output = image.copy()
    # loop over the current subset of region proposals
    for (x, y, w, h) in rects[i:i + 100]:
        # draw the region proposal bounding box on the image
        color = [random.randint(0, 255) for j in range(0, 3)]
        cv2.rectangle(output, (x, y), (x + w, y + h), color, 2)
    # show the output image
    cv2.imshow("Output", output)
    key = cv2.waitKey(0) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break